### Lecture May 28
[link](https://umd.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=860725a8-12f3-4c1a-8676-ae850110107a)

### Trait Bounds
- parameterize over a type that has a trait

```
pub fn notify<T: Summarizable>(item: T) {
	println("{}",item.summary());
}
```

- same as in Java... `<T extends Summarizable>`
- where `T` must implement `Summarizable`

- trait implementations can be generic 6:30

- multiple trait bounds using +
`fn foo<T: Clone + Summarizable>() -> i32 {}` or `fn foo<T>() -> i32 where T: Clone + Summarizable {}`



- `Box<T>` Smart Pointers

- `Box<T>` values point to heap-allocated data
- the `Box<T>` value (the pointer) is on the stack, while its pointed-to T value is allocated on the heap
	- has `Deref` trait
	- has `Drop` trait

- can reduce copying?
- more efficient recursive type?

Example: a Linked List

this doesn't work:
```
enum List {
	Nul,
	Cons(i32,List)
}
```
- compiler doesn't know the size of List
- since List is recursive, it could be any size

soln: use a Box to add an indirection...

```
enum List {
	Nul, Cons(i32, Box<List>)
}
```
now the size is fixed: i32 + size of pointer OR size of Nil


"referred-to" data (in heap) dropped as soon as the box goes out of scope

Deref

- if `x` is an `int` then `&x` is an `&int`

- `*(&x) = x`

- Deref trait requires `deref(&self) -> &T` method

- DerefMut trait for when an object is mutable

- example of compiler inserting a deref call 22:22

receiver method 27:00
- Java has dynamic dispatch (polymorphism). The correct method to run is determined at runtime

- To implement dispatch in Rust, use trait objects

- `dyn Summarizable`, wrapped in `Box` 30:00

```
fn print_summarizable(s: Box<dyn Summarizable>) {
	println!("{}", sm.summary())
}
```

callers of method use Box to put the data on the heap

```
pub fn main() {
	let b = Box::new(42);
	print_summarizable(b)
}
```


- need Box because `dyn Summarizable` doesn't have a known size at compile-time

Box and size
- `Box<i32>` is a pointer to a heap-allocated i32
- `Box<dyn Summarizable>` is a fat pointer to a heap-allocated Summarizable

size of struct 33:30




Box: smart pointer 34:53
- A smart pointer is a reference + metadata
- `String`, `Vec<T>`, `Box<T>`

Boxes are useful for non cyclic, immutable data structures
use trait objects of type `Box<dyn Trait>`

Rust ownership / mutability rules 37:55
- enables type safety, but quite restrictive
- sometimes mutation and sharing is useful
- shared mutable access to the owner is disallowed in rust
- there is a way to overcome this
	- multiple references to mutable data, but, keep track of how many
	- this is called interior mutability
	- there are extra space and time overhead when keeping track of references
	- might violate rust ownership at runtime

example 49:10


rules for interior mutability 53:25
- smart pointer associates a counter with the underlying reference
- clone copes the pointer and bumps the counter by one
- calling drop reduces the counter by one
- when the counter hits zero the data is freed
- counter is called refCount

cyclic data 1:05:00

Rc references: no mutation 1:09:42
- needed for a case like lists with tails that are shared
- cant have multiple references that mutate the heap value

- `Box<T>`references data that is immutable
- `Cell<T>` is like `Box<T>` but with mutable contents
- Rc is like `Box<T>` but with multiple references
- RefCell is like `Cell<T>` but with multiple mutable references

RefCell has a *dynamic* borrow